In [112]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os
import requests

In [113]:
load_dotenv()
HF_API_KEY = os.getenv('HF_KEY')

llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-V3-0324",
    api_key=HF_API_KEY,
    base_url="https://router.huggingface.co/v1",
    temperature=0.7,
    max_tokens=500
)

In [114]:
response = llm.invoke('what is capital of pakistan').content
response

'The capital of **Pakistan** is **Islamabad**.  \n\nIslamabad became the capital in **1967**, replacing **Karachi**. It is located in the northern part of the country, near the Margalla Hills. The city is known for its modern planning, green spaces, and important government buildings, including the **President House** and **Parliament of Pakistan**.  \n\nWould you like more details about Islamabad? 😊'

In [115]:
from langchain_core.tools import tool

In [116]:
@tool
def multiply(a:int, b:int) ->int:
    "multiply the two number"
    return a*b

In [117]:
llm_with_tools = llm.bind_tools([multiply])

In [118]:
query = HumanMessage(content='what is the product of 7 and 8?')

messages = [query]
messages

[HumanMessage(content='what is the product of 7 and 8?', additional_kwargs={}, response_metadata={})]

In [119]:
response = llm_with_tools.invoke('what is the product of 7 and 8?')
messages.append(response)
result = response.tool_calls[0]
print(result)


{'name': 'multiply', 'args': {'a': 7, 'b': 8}, 'id': 'chatcmpl-tool-85245abbce5040aca4c2b231dca476bd', 'type': 'tool_call'}


In [120]:
ans = multiply.invoke(result)
messages.append(ans)
ans

ToolMessage(content='56', name='multiply', tool_call_id='chatcmpl-tool-85245abbce5040aca4c2b231dca476bd')

In [121]:
messages

[HumanMessage(content='what is the product of 7 and 8?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 240, 'total_tokens': 285, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'deepseek/deepseek-v3-0324', 'system_fingerprint': '', 'id': '93a14fe82d1bc5bf386297a0bc921b99', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7612-8111-70a1-8289-ff1a4509dca0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 7, 'b': 8}, 'id': 'chatcmpl-tool-85245abbce5040aca4c2b231dca476bd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 240, 'output_tokens': 45, 'total_tokens': 285, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='56', name='multiply', tool_call_id='chatcmpl-tool-85245abbce5040aca4c2b231dca476bd')]

In [122]:
final_result = llm.invoke(messages).content
print(final_result)

The product of 7 and 8 is **56**.  

Here's the calculation:  
\[ 7 \times 8 = 56 \]  

Let me know if you'd like help with anything else!


In [123]:
@tool
def get_weather_data(city:str) ->str:
    """
    Returns current weather for a given city
    """
    url = f'https://api.weatherapi.com/v1/current.json?key=3438e9fbe45b445bbc772347253012&q={city}&aqi=yes'
    response = requests.get(url)
    return response.json()

In [124]:
get_weather_data.invoke({"city":'karachi'})

{'location': {'name': 'Karachi',
  'region': 'Sindh',
  'country': 'Pakistan',
  'lat': 24.8667,
  'lon': 67.05,
  'tz_id': 'Asia/Karachi',
  'localtime_epoch': 1767212524,
  'localtime': '2026-01-01 01:22'},
 'current': {'last_updated_epoch': 1767212100,
  'last_updated': '2026-01-01 01:15',
  'temp_c': 15.1,
  'temp_f': 59.2,
  'is_day': 0,
  'air_quality': {'co': 1795.85,
   'no2': 42.65,
   'o3': 46.0,
   'so2': 15.45,
   'pm2_5': 23.05,
   'pm10': 37.55,
   'us-epa-index': 2,
   'gb-defra-index': 2}}}

In [125]:
# second step : Binding with llm
llm_with_tool = llm.bind_tools([get_weather_data])

In [126]:
# Tool calling 
messages = [HumanMessage('tell me the current weather of bahawalpur')]
messages

[HumanMessage(content='tell me the current weather of bahawalpur', additional_kwargs={}, response_metadata={})]

In [127]:
ai_messages = llm_with_tool.invoke(messages)
messages.append(ai_messages)
messages
ai_messages.tool_calls[0]

{'name': 'get_weather_data',
 'args': {'city': 'bahawalpur'},
 'id': '527470db6c664cca84b4fb90e4985aa8',
 'type': 'tool_call'}

In [128]:
tool_messages = get_weather_data.invoke(ai_messages.tool_calls[0])
messages.append(tool_messages)
tool_messages 

ToolMessage(content='{"location": {"name": "Bahawalpur", "region": "Punjab", "country": "Pakistan", "lat": 29.4, "lon": 71.6833, "tz_id": "Asia/Karachi", "localtime_epoch": 1767212314, "localtime": "2026-01-01 01:18"}, "current": {"last_updated_epoch": 1767212100, "last_updated": "2026-01-01 01:15", "temp_c": 14.7, "temp_f": 58.5, "is_day": 0, "air_quality": {"co": 1459.85, "no2": 16.15, "o3": 83.0, "so2": 16.75, "pm2_5": 90.95, "pm10": 92.55, "us-epa-index": 4, "gb-defra-index": 10}}}', name='get_weather_data', tool_call_id='527470db6c664cca84b4fb90e4985aa8')

In [129]:
messages

[HumanMessage(content='tell me the current weather of bahawalpur', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 400, 'total_tokens': 446, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 384, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'text_tokens': 0, 'image_tokens': 0, 'video_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'deepseek/deepseek-v3-0324', 'system_fingerprint': '', 'id': '3077fd78e2688adfc42f14f0922ea81b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7612-9f00-70a2-85df-c35c8b65ff2b-0', tool_calls=[{'name': 'get_weather_data', 'args': {'city': 'bahawalpur'}, 'id': '527470db6c664cca84b4fb90e4985aa8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 400, 'output_tokens': 46, 'total_tokens': 446, 'input_token_details': {'audio': 0, 'cache

In [130]:
result =llm.invoke(messages)
print(result.content)

Here’s the current weather in **Bahawalpur, Pakistan** (as of the latest available data):  

- **Temperature**: **14.7°C (58.5°F)**  
- **Condition**: Nighttime (no daylight)  
- **Air Quality**: **Unhealthy** (PM2.5: 90.95 µg/m³, US-EPA Index: 4)  

For real-time updates, you may check a reliable weather service like [AccuWeather](https://www.accuweather.com/) or [Weather.com](https://weather.com/). Let me know if you'd like help interpreting the data further!
